In [1]:
using StaticArrays
using ForwardDiff

INFO: Recompiling stale cache file /home/kylebrown/.julia/lib/v0.6/ForwardDiff.ji for module ForwardDiff.


In [2]:
import ForwardDiff: dualize, Chunk, Dual

# Accessing length() and eltype() directly from type parameters

In [3]:
# # @generated function dualize2(::Type{T}, x::SArray{S,V,D,N}) where {T,S,V,D,N}
# @generated function dualize2(::Type{T}, x::Union{FieldVector{N,V}, SArray{S,V,D,N}}) where {T,S,V,D,N}
# #     N = length(x)
# #     V = eltype(x)
#     dx = Expr(:tuple, [:(Dual{T}(x[$i], chunk, Val{$i}())) for i in 1:N]...)
#     return quote
#         chunk = Chunk{N}()
#         $(Expr(:meta, :inline))
#         # return SArray{S}($(dx))
#         similar_type($x,Dual{T,V,N})($(dx))
#     end
# end

In [4]:
p = SVector(1.0,2.0)

2-element SVector{2,Float64}:
 1.0
 2.0

In [5]:
dualize2(Float64, p)

LoadError: [91mUndefVarError: dualize2 not defined[39m

# AutomaticDifferentiation

In [6]:
struct Point{R<:Real} <: FieldVector{2,R}
    x::R
    y::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(2,)}) where {P<:Point, R<:Real} = Point{R}

In [7]:
x = Point(1.0,2.0)

2-element Point{Float64}:
 1.0
 2.0

In [8]:
PT = typeof(x)
ST = typeof(SVector(1.0,2.0))

SVector{2,Float64}

In [9]:
N,V = supertype(typeof(x)).parameters
S,V,D,N = typeof(SVector(1.0,2.0)).parameters

svec(Tuple{2}, Float64, 1, 2)

In [10]:
S,V,D,N = typeof(SVector(1.0,2.0)).parameters

svec(Tuple{2}, Float64, 1, 2)

In [11]:
SN,SV, = ST.parameters

svec(Tuple{2}, Float64, 1, 2)

In [12]:
p = Point(1.0,2.0)

2-element Point{Float64}:
 1.0
 2.0

In [13]:
p .- Point(3.0,5.0)

2-element Point{Float64}:
 -2.0
 -3.0

In [14]:
p = Point(2.0,3.0)
x = SVector(2.0,3.0)
dualize(Float64, p)

2-element Point{ForwardDiff.Dual{Float64,Float64,2}}:
 Dual{Float64}(2.0,1.0,0.0)
 Dual{Float64}(3.0,0.0,1.0)

In [15]:
dualize(Float64,x)

2-element SVector{2,ForwardDiff.Dual{Float64,Float64,2}}:
 Dual{Float64}(2.0,1.0,0.0)
 Dual{Float64}(3.0,0.0,1.0)

In [16]:
f(p) = p.x
ForwardDiff.gradient(f, Point(2.0,3.0))

2-element Point{Float64}:
 1.0
 0.0

In [17]:
f(p) = sum(p)
ForwardDiff.gradient(f, SVector(1,2,3))

3-element SVector{3,Int64}:
 1
 1
 1

In [18]:
A = SArray{Tuple{10,1}}(ones(10))
A_d = ForwardDiff.dualize(Float64, A)

10×1 StaticArrays.SArray{Tuple{10,1},ForwardDiff.Dual{Float64,Float64,10},2,10}:
 Dual{Float64}(1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0)
 Dual{Float64}(1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0)

In [19]:
A = [1.0;2.0]
P = Point(1.0,2.0)
A_d = ForwardDiff.dualize(Float64, P)

2-element Point{ForwardDiff.Dual{Float64,Float64,2}}:
 Dual{Float64}(1.0,1.0,0.0)
 Dual{Float64}(2.0,0.0,1.0)

# Define generated dualize() method for FieldVectors

In [20]:
p = Point(2.0,3.0)
dualize(Float64, p)

2-element Point{ForwardDiff.Dual{Float64,Float64,2}}:
 Dual{Float64}(2.0,1.0,0.0)
 Dual{Float64}(3.0,0.0,1.0)

# Gradient

In [21]:
x = rand(2, 1)
fx = Point(x[1],x[2]) # Point
sx = StaticArrays.SArray{Tuple{2,1}}(x)

cfg = ForwardDiff.GradientConfig(nothing, x)
fcfg = ForwardDiff.GradientConfig(nothing, fx)
scfg = ForwardDiff.GradientConfig(nothing, sx);

In [22]:
import ForwardDiff: extract_gradient, partials, vector_mode_gradient, Tag, Partials, extract_gradient!

In [23]:
@show extract_gradient!(Float64, similar(x), 1.0)
@show extract_gradient(Float64, 1.0, sx)
@show extract_gradient(Float64, 1.0, fx)

extract_gradient!(Float64, similar(x), 1.0) = [0.0; 0.0]
extract_gradient(Float64, 1.0, sx) = [0.0; 0.0]
extract_gradient(Float64, 1.0, fx) = [0.0, 0.0]


2-element Point{Float64}:
 0.0
 0.0

In [24]:
d = Dual{2}(1.0,Partials((1.0,2.0)))
partials(d,2)
partials(1.0,d,2)

0.0

In [25]:
@show vector_mode_gradient(prod, x, cfg)
@show vector_mode_gradient(prod, sx, scfg)

vector_mode_gradient(prod, x, cfg) = [0.59146; 0.0622996]
vector_mode_gradient(prod, sx, scfg) = 

2×1 StaticArrays.MArray{Tuple{2,1},Float64,2,2}:
 0.59146  
 0.0622996

[0.59146; 0.0622996]


In [26]:
@show vector_mode_gradient(prod, fx, fcfg)

vector_mode_gradient(prod, fx, fcfg) = [0.59146, 0.0622996]


2-element MVector{2,Float64}:
 0.59146  
 0.0622996

In [27]:
@show vector_mode_gradient(prod, fx)

vector_mode_gradient(prod, fx) = [0.59146, 0.0622996]


2-element Point{Float64}:
 0.59146  
 0.0622996

# Need to implement GradientConfig() for SArray and FieldVector

In [28]:
Point(1.0,2.0) == SArray(Point(1.0,2.0))

true

In [29]:
@show ForwardDiff.gradient(prod, fx)
@show ForwardDiff.gradient(p -> p.x*p.y, fx)

ForwardDiff.gradient(prod, fx) = [0.59146, 0.0622996]
ForwardDiff.gradient((p->begin  # In[29], line 2:
            p.x * p.y
        end), fx) = [0.59146, 0.0622996]


2-element Point{Float64}:
 0.59146  
 0.0622996

In [30]:
@show ForwardDiff.gradient(prod, x, cfg)[:] == ForwardDiff.gradient(prod, fx, fcfg)[:]

(ForwardDiff.gradient(prod, x, cfg))[:] == (ForwardDiff.gradient(prod, fx, fcfg))[:] = true


true

# Hessian

In [31]:
ForwardDiff.hessian(p -> (p.x*p.y)^2, fx)

2×2 StaticArrays.SArray{Tuple{2,2},Float64,2,4}:
 0.699649  0.147391  
 0.147391  0.00776248

In [32]:
struct BigPoint{R <: Real} <: FieldVector{5,R}
    x::R
    y::R
    z::R
    θ::R
    s::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(5,)}) where {P<:BigPoint, R<:Real} = BigPoint{R}

In [33]:
ForwardDiff.hessian(p -> (p.x*p.y)^2, Point(1.0,2.0))

2×2 StaticArrays.SArray{Tuple{2,2},Float64,2,4}:
 8.0  8.0
 8.0  2.0

In [34]:
P = BigPoint(1.0,2.0,3.0,π/4,100.0)

5-element BigPoint{Float64}:
   1.0     
   2.0     
   3.0     
   0.785398
 100.0     

In [35]:
ForwardDiff.hessian(p -> (p.x*p.y + p.z + p.s*sin(p.θ))^2, P)

5×5 StaticArrays.SArray{Tuple{5,5},Float64,2,25}:
   8.0      155.421      4.0       282.843    2.82843
 155.421      2.0        2.0       141.421    1.41421
   4.0        2.0        2.0       141.421    1.41421
 282.843    141.421    141.421    -707.107  207.071  
   2.82843    1.41421    1.41421   207.071    1.0    

# Tests

# Gradient Tests for Static Arrays

In [36]:
struct Point3D{R<:Real} <: FieldVector{3,R}
    x::R
    y::R
    z::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(3,)}) where {P<:Point3D, R<:Real} = Point3D{R}

In [37]:
println("  ...testing specialized StaticArray codepaths")

x = rand(3, 3)
sx = StaticArrays.SArray{Tuple{3,3}}(x)

cfg = ForwardDiff.GradientConfig(nothing, x)
scfg = ForwardDiff.GradientConfig(nothing, sx)

actual = ForwardDiff.gradient(prod, x)
@show ForwardDiff.gradient(prod, sx) == actual
@show ForwardDiff.gradient(prod, sx, cfg) == actual
@show ForwardDiff.gradient(prod, sx, scfg) == actual

out = similar(x)
ForwardDiff.gradient!(out, prod, sx)
@show out == actual

out = similar(x)
ForwardDiff.gradient!(out, prod, sx, cfg)
@show out == actual

out = similar(x)
ForwardDiff.gradient!(out, prod, sx, scfg)
@show out == actual

result = DiffResults.GradientResult(x)
result = ForwardDiff.gradient!(result, prod, x)

result1 = DiffResults.GradientResult(x)
result2 = DiffResults.GradientResult(x)
result3 = DiffResults.GradientResult(x)
result1 = ForwardDiff.gradient!(result1, prod, sx)
result2 = ForwardDiff.gradient!(result2, prod, sx, cfg)
result3 = ForwardDiff.gradient!(result3, prod, sx, scfg)
@show DiffResults.value(result1) == DiffResults.value(result)
@show DiffResults.value(result2) == DiffResults.value(result)
@show DiffResults.value(result3) == DiffResults.value(result)
@show DiffResults.gradient(result1) == DiffResults.gradient(result)
@show DiffResults.gradient(result2) == DiffResults.gradient(result)
@show DiffResults.gradient(result3) == DiffResults.gradient(result)

sresult1 = DiffResults.GradientResult(sx)
sresult2 = DiffResults.GradientResult(sx)
sresult3 = DiffResults.GradientResult(sx)
sresult1 = ForwardDiff.gradient!(sresult1, prod, sx)
sresult2 = ForwardDiff.gradient!(sresult2, prod, sx, cfg)
sresult3 = ForwardDiff.gradient!(sresult3, prod, sx, scfg)
@show DiffResults.value(sresult1) == DiffResults.value(result)
@show DiffResults.value(sresult2) == DiffResults.value(result)
@show DiffResults.value(sresult3) == DiffResults.value(result)
@show DiffResults.gradient(sresult1) == DiffResults.gradient(result)
@show DiffResults.gradient(sresult2) == DiffResults.gradient(result)
@show DiffResults.gradient(sresult3) == DiffResults.gradient(result)

  ...testing specialized StaticArray codepaths
ForwardDiff.gradient(prod, sx) == actual = true
ForwardDiff.gradient(prod, sx, cfg) == actual = true
ForwardDiff.gradient(prod, sx, scfg) == actual = true
out == actual = true
out == actual = true
out == actual = true
DiffResults.value(result1) == DiffResults.value(result) = true
DiffResults.value(result2) == DiffResults.value(result) = true
DiffResults.value(result3) == DiffResults.value(result) = true
DiffResults.gradient(result1) == DiffResults.gradient(result) = true
DiffResults.gradient(result2) == DiffResults.gradient(result) = true
DiffResults.gradient(result3) == DiffResults.gradient(result) = true
DiffResults.value(sresult1) == DiffResults.value(result) = true
DiffResults.value(sresult2) == DiffResults.value(result) = true
DiffResults.value(sresult3) == DiffResults.value(result) = true
DiffResults.gradient(sresult1) == DiffResults.gradient(result) = true
DiffResults.gradient(sresult2) == DiffResults.gradient(result) = true
DiffRes

true

In [38]:
println("  ...testing specialized FieldVector codepaths")

struct Point3D{R<:Real} <: FieldVector{3,R}
    x::R
    y::R
    z::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(3,)}) where {P<:Point3D, R<:Real} = Point3D{R}

x = rand(3, 1)
fx = Point3D(x)

cfg = ForwardDiff.GradientConfig(nothing, x)
fcfg = ForwardDiff.GradientConfig(nothing, fx)

actual = ForwardDiff.gradient(prod, x)
@show ForwardDiff.gradient(prod, fx) == actual[:]
@show ForwardDiff.gradient(prod, fx, cfg) == actual[:]
@show ForwardDiff.gradient(prod, fx, fcfg) == actual[:]

out = similar(x)
ForwardDiff.gradient!(out, prod, fx)
@show out == actual

out = similar(x)
ForwardDiff.gradient!(out, prod, fx, cfg)
@show out == actual

out = similar(x)
ForwardDiff.gradient!(out, prod, fx, fcfg)
@show out == actual

result = DiffResults.GradientResult(x)
result = ForwardDiff.gradient!(result, prod, x)

result1 = DiffResults.GradientResult(x)
result2 = DiffResults.GradientResult(x)
result3 = DiffResults.GradientResult(x)
result1 = ForwardDiff.gradient!(result1, prod, fx)
result2 = ForwardDiff.gradient!(result2, prod, fx, cfg)
result3 = ForwardDiff.gradient!(result3, prod, fx, fcfg)
@show DiffResults.value(result1) == DiffResults.value(result)
@show DiffResults.value(result2) == DiffResults.value(result)
@show DiffResults.value(result3) == DiffResults.value(result)
@show DiffResults.gradient(result1) == DiffResults.gradient(result)
@show DiffResults.gradient(result2) == DiffResults.gradient(result)
@show DiffResults.gradient(result3) == DiffResults.gradient(result)

fresult1 = DiffResults.GradientResult(fx)
fresult2 = DiffResults.GradientResult(fx)
fresult3 = DiffResults.GradientResult(fx)
fresult1 = ForwardDiff.gradient!(fresult1, prod, fx)
fresult2 = ForwardDiff.gradient!(fresult2, prod, fx, cfg)
fresult3 = ForwardDiff.gradient!(fresult3, prod, fx, fcfg)
@show DiffResults.value(fresult1) == DiffResults.value(result)
@show DiffResults.value(fresult2) == DiffResults.value(result)
@show DiffResults.value(fresult3) == DiffResults.value(result)
@show DiffResults.gradient(fresult1) == DiffResults.gradient(result)[:]
@show DiffResults.gradient(fresult2) == DiffResults.gradient(result)[:]
@show DiffResults.gradient(fresult3) == DiffResults.gradient(result)[:]

  ...testing specialized FieldVector codepaths
ForwardDiff.gradient(prod, fx) == actual[:] = true
ForwardDiff.gradient(prod, fx, cfg) == actual[:] = true
ForwardDiff.gradient(prod, fx, fcfg) == actual[:] = true
out == actual = true
out == actual = true
out == actual = true
DiffResults.value(result1) == DiffResults.value(result) = true
DiffResults.value(result2) == DiffResults.value(result) = true
DiffResults.value(result3) == DiffResults.value(result) = true
DiffResults.gradient(result1) == DiffResults.gradient(result) = true
DiffResults.gradient(result2) == DiffResults.gradient(result) = true
DiffResults.gradient(result3) == DiffResults.gradient(result) = true
DiffResults.value(fresult1) == DiffResults.value(result) = true
DiffResults.value(fresult2) == DiffResults.value(result) = true
DiffResults.value(fresult3) == DiffResults.value(result) = true
DiffResults.gradient(fresult1) == (DiffResults.gradient(result))[:] = true
DiffResults.gradient(fresult2) == (DiffResults.gradient(result)

true

# Hessian Tests for Static Arrays

In [39]:
x = rand(3, 3)
sx = StaticArrays.SArray{Tuple{3,3}}(x)

cfg = ForwardDiff.HessianConfig(nothing, x)
scfg = ForwardDiff.HessianConfig(nothing, sx)

actual = ForwardDiff.hessian(prod, x)
@show ForwardDiff.hessian(prod, sx) == actual
@show ForwardDiff.hessian(prod, sx, cfg) == actual
@show ForwardDiff.hessian(prod, sx, scfg) == actual

out = similar(x, 9, 9)
ForwardDiff.hessian!(out, prod, sx)
@show out == actual

out = similar(x, 9, 9)
ForwardDiff.hessian!(out, prod, sx, cfg)
@show out == actual

out = similar(x, 9, 9)
ForwardDiff.hessian!(out, prod, sx, scfg)
@show out == actual

result = DiffResults.HessianResult(x)
result = ForwardDiff.hessian!(result, prod, x)

result1 = DiffResults.HessianResult(x)
result2 = DiffResults.HessianResult(x)
result3 = DiffResults.HessianResult(x)
result1 = ForwardDiff.hessian!(result1, prod, sx)
result2 = ForwardDiff.hessian!(result2, prod, sx, ForwardDiff.HessianConfig(prod, result2, x, ForwardDiff.Chunk(x), nothing))
result3 = ForwardDiff.hessian!(result3, prod, sx, ForwardDiff.HessianConfig(prod, result3, x, ForwardDiff.Chunk(x), nothing))
@show DiffResults.value(result1) == DiffResults.value(result)
@show DiffResults.value(result2) == DiffResults.value(result)
@show DiffResults.value(result3) == DiffResults.value(result)
@show DiffResults.gradient(result1) == DiffResults.gradient(result)
@show DiffResults.gradient(result2) == DiffResults.gradient(result)
@show DiffResults.gradient(result3) == DiffResults.gradient(result)
@show DiffResults.hessian(result1) == DiffResults.hessian(result)
@show DiffResults.hessian(result2) == DiffResults.hessian(result)
@show DiffResults.hessian(result3) == DiffResults.hessian(result)

sresult1 = DiffResults.HessianResult(sx)
sresult2 = DiffResults.HessianResult(sx)
sresult3 = DiffResults.HessianResult(sx)
sresult1 = ForwardDiff.hessian!(sresult1, prod, sx)
sresult2 = ForwardDiff.hessian!(sresult2, prod, sx, ForwardDiff.HessianConfig(prod, sresult2, x, ForwardDiff.Chunk(x), nothing))
sresult3 = ForwardDiff.hessian!(sresult3, prod, sx, ForwardDiff.HessianConfig(prod, sresult3, x, ForwardDiff.Chunk(x), nothing))
@show DiffResults.value(sresult1) == DiffResults.value(result)
@show DiffResults.value(sresult2) == DiffResults.value(result)
@show DiffResults.value(sresult3) == DiffResults.value(result)
@show DiffResults.gradient(sresult1) == DiffResults.gradient(result)
@show DiffResults.gradient(sresult2) == DiffResults.gradient(result)
@show DiffResults.gradient(sresult3) == DiffResults.gradient(result)
@show DiffResults.hessian(sresult1) == DiffResults.hessian(result)
@show DiffResults.hessian(sresult2) == DiffResults.hessian(result)
@show DiffResults.hessian(sresult3) == DiffResults.hessian(result)

ForwardDiff.hessian(prod, sx) == actual = true
ForwardDiff.hessian(prod, sx, cfg) == actual = true
ForwardDiff.hessian(prod, sx, scfg) == actual = true
out == actual = true
out == actual = true
out == actual = true
DiffResults.value(result1) == DiffResults.value(result) = true
DiffResults.value(result2) == DiffResults.value(result) = true
DiffResults.value(result3) == DiffResults.value(result) = true
DiffResults.gradient(result1) == DiffResults.gradient(result) = true
DiffResults.gradient(result2) == DiffResults.gradient(result) = true
DiffResults.gradient(result3) == DiffResults.gradient(result) = true
DiffResults.hessian(result1) == DiffResults.hessian(result) = true
DiffResults.hessian(result2) == DiffResults.hessian(result) = true
DiffResults.hessian(result3) == DiffResults.hessian(result) = true
DiffResults.value(sresult1) == DiffResults.value(result) = true
DiffResults.value(sresult2) == DiffResults.value(result) = true
DiffResults.value(sresult3) == DiffResults.value(result) = t

true

# Hessian tests for field vector

In [40]:
println("  ...testing specialized FieldVector codepaths")

struct Point3D{R<:Real} <: FieldVector{3,R}
    x::R
    y::R
    z::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(3,)}) where {P<:Point3D, R<:Real} = Point3D{R}

x = rand(3, 1)
fx = Point3D(x)

cfg = ForwardDiff.HessianConfig(nothing, x)
fcfg = ForwardDiff.HessianConfig(nothing, fx)

actual = ForwardDiff.hessian(prod, x)
@test ForwardDiff.hessian(prod, fx) == actual
@test ForwardDiff.hessian(prod, fx, cfg) == actual
@test ForwardDiff.hessian(prod, fx, fcfg) == actual

out = similar(x, length(x), length(x))
ForwardDiff.hessian!(out, prod, fx)
@test out == actual

out = similar(x, length(x), length(x))
ForwardDiff.hessian!(out, prod, fx, cfg)
@test out == actual

out = similar(x, length(x), length(x))
ForwardDiff.hessian!(out, prod, fx, fcfg)
@test out == actual

result = DiffResults.HessianResult(x)
result = ForwardDiff.hessian!(result, prod, x)

result1 = DiffResults.HessianResult(x)
result2 = DiffResults.HessianResult(x)
result3 = DiffResults.HessianResult(x)
result1 = ForwardDiff.hessian!(result1, prod, fx)
result2 = ForwardDiff.hessian!(result2, prod, fx, ForwardDiff.HessianConfig(prod, result2, x, ForwardDiff.Chunk(x), nothing))
result3 = ForwardDiff.hessian!(result3, prod, fx, ForwardDiff.HessianConfig(prod, result3, x, ForwardDiff.Chunk(x), nothing))
@test DiffResults.value(result1) == DiffResults.value(result)
@test DiffResults.value(result2) == DiffResults.value(result)
@test DiffResults.value(result3) == DiffResults.value(result)
@test DiffResults.gradient(result1) == DiffResults.gradient(result)
@test DiffResults.gradient(result2) == DiffResults.gradient(result)
@test DiffResults.gradient(result3) == DiffResults.gradient(result)
@test DiffResults.hessian(result1) == DiffResults.hessian(result)
@test DiffResults.hessian(result2) == DiffResults.hessian(result)
@test DiffResults.hessian(result3) == DiffResults.hessian(result)

fresult1 = DiffResults.HessianResult(fx)
fresult2 = DiffResults.HessianResult(fx)
fresult3 = DiffResults.HessianResult(fx)
fresult1 = ForwardDiff.hessian!(fresult1, prod, fx)
fresult2 = ForwardDiff.hessian!(fresult2, prod, fx, ForwardDiff.HessianConfig(prod, fresult2, x, ForwardDiff.Chunk(x), nothing))
fresult3 = ForwardDiff.hessian!(fresult3, prod, fx, ForwardDiff.HessianConfig(prod, fresult3, x, ForwardDiff.Chunk(x), nothing))
@test DiffResults.value(fresult1) == DiffResults.value(result)
@test DiffResults.value(fresult2) == DiffResults.value(result)
@test DiffResults.value(fresult3) == DiffResults.value(result)
@test DiffResults.gradient(fresult1) == DiffResults.gradient(result)[:]
@test DiffResults.gradient(fresult2) == DiffResults.gradient(result)[:]
@test DiffResults.gradient(fresult3) == DiffResults.gradient(result)[:]
@test DiffResults.hessian(fresult1) == DiffResults.hessian(result)
@test DiffResults.hessian(fresult2) == DiffResults.hessian(result)
@test DiffResults.hessian(fresult3) == DiffResults.hessian(result)

  ...testing specialized FieldVector codepaths


LoadError: [91mUndefVarError: @test not defined[39m

# Jacobian Tests for SArray

In [41]:
println("  ...testing specialized FieldVector codepaths")

x = rand(3, 3)
sx = StaticArrays.SArray{Tuple{3,3}}(x)

cfg = ForwardDiff.JacobianConfig(nothing, x)
scfg = ForwardDiff.JacobianConfig(nothing, sx)

actual = ForwardDiff.jacobian(diff, x)
@show ForwardDiff.jacobian(diff, sx) == actual
@show ForwardDiff.jacobian(diff, sx, cfg) == actual
@show ForwardDiff.jacobian(diff, sx, scfg) == actual

out = similar(x, 6, 9)
ForwardDiff.jacobian!(out, diff, sx)
@show out == actual

out = similar(x, 6, 9)
ForwardDiff.jacobian!(out, diff, sx, cfg)
@show out == actual

out = similar(x, 6, 9)
ForwardDiff.jacobian!(out, diff, sx, scfg)
@show out == actual

result = DiffResults.JacobianResult(similar(x, 6), x)
result = ForwardDiff.jacobian!(result, diff, x)

result1 = DiffResults.JacobianResult(similar(sx, 6), sx)
result2 = DiffResults.JacobianResult(similar(sx, 6), sx)
result3 = DiffResults.JacobianResult(similar(sx, 6), sx)
result1 = ForwardDiff.jacobian!(result1, diff, sx)
result2 = ForwardDiff.jacobian!(result2, diff, sx, cfg)
result3 = ForwardDiff.jacobian!(result3, diff, sx, scfg)
@show DiffResults.value(result1) == DiffResults.value(result)
@show DiffResults.value(result2) == DiffResults.value(result)
@show DiffResults.value(result3) == DiffResults.value(result)
@show DiffResults.jacobian(result1) == DiffResults.jacobian(result)
@show DiffResults.jacobian(result2) == DiffResults.jacobian(result)
@show DiffResults.jacobian(result3) == DiffResults.jacobian(result)

sy = @SVector fill(zero(eltype(sx)), 6)
sresult1 = DiffResults.JacobianResult(sy, sx)
sresult2 = DiffResults.JacobianResult(sy, sx)
sresult3 = DiffResults.JacobianResult(sy, sx)
sresult1 = ForwardDiff.jacobian!(sresult1, diff, sx)
sresult2 = ForwardDiff.jacobian!(sresult2, diff, sx, cfg)
sresult3 = ForwardDiff.jacobian!(sresult3, diff, sx, scfg)
@show DiffResults.value(sresult1) == DiffResults.value(result)
@show DiffResults.value(sresult2) == DiffResults.value(result)
@show DiffResults.value(sresult3) == DiffResults.value(result)
@show DiffResults.jacobian(sresult1) == DiffResults.jacobian(result)
@show DiffResults.jacobian(sresult2) == DiffResults.jacobian(result)
@show DiffResults.jacobian(sresult3) == DiffResults.jacobian(result)

  ...testing specialized FieldVector codepaths
ForwardDiff.jacobian(diff, sx) == actual = true
ForwardDiff.jacobian(diff, sx, cfg) == actual = true
ForwardDiff.jacobian(diff, sx, scfg) == actual = true
out == actual = true
out == actual = true
out == actual = true
DiffResults.value(result1) == DiffResults.value(result) = true
DiffResults.value(result2) == DiffResults.value(result) = true
DiffResults.value(result3) == DiffResults.value(result) = true
DiffResults.jacobian(result1) == DiffResults.jacobian(result) = true
DiffResults.jacobian(result2) == DiffResults.jacobian(result) = true
DiffResults.jacobian(result3) == DiffResults.jacobian(result) = true
DiffResults.value(sresult1) == DiffResults.value(result) = true
DiffResults.value(sresult2) == DiffResults.value(result) = true
DiffResults.value(sresult3) == DiffResults.value(result) = true
DiffResults.jacobian(sresult1) == DiffResults.jacobian(result) = true
DiffResults.jacobian(sresult2) == DiffResults.jacobian(result) = true
DiffRes

true

In [42]:
println("  ...testing specialized FieldVector codepaths")

struct Point3D{R<:Real} <: FieldVector{3,R}
    x::R
    y::R
    z::R
end
StaticArrays.similar_type(p::Type{P}, ::Type{R}, size::Size{(3,)}) where {P<:Point3D, R<:Real} = Point3D{R}

x = rand(3, 1)
fx = Point3D(x)

cfg = ForwardDiff.JacobianConfig(nothing, x)
fcfg = ForwardDiff.JacobianConfig(nothing, fx)

actual = ForwardDiff.jacobian(diff, x)
@test ForwardDiff.jacobian(diff, fx) == actual
@test ForwardDiff.jacobian(diff, fx, cfg) == actual
@test ForwardDiff.jacobian(diff, fx, fcfg) == actual

out = similar(x, 2, 3)
ForwardDiff.jacobian!(out, diff, fx)
@test out == actual

out = similar(x, 2, 3)
ForwardDiff.jacobian!(out, diff, fx, cfg)
@test out == actual

out = similar(x, 2, 3)
ForwardDiff.jacobian!(out, diff, fx, fcfg)
@test out == actual

result = DiffResults.JacobianResult(similar(x, 2), x)
result = ForwardDiff.jacobian!(result, diff, x)

result1 = DiffResults.JacobianResult(similar(fx, 2), fx)
result2 = DiffResults.JacobianResult(similar(fx, 2), fx)
result3 = DiffResults.JacobianResult(similar(fx, 2), fx)
result1 = ForwardDiff.jacobian!(result1, diff, fx)
result2 = ForwardDiff.jacobian!(result2, diff, fx, cfg)
result3 = ForwardDiff.jacobian!(result3, diff, fx, fcfg)
@test DiffResults.value(result1) == DiffResults.value(result)
@test DiffResults.value(result2) == DiffResults.value(result)
@test DiffResults.value(result3) == DiffResults.value(result)
@test DiffResults.jacobian(result1) == DiffResults.jacobian(result)
@test DiffResults.jacobian(result2) == DiffResults.jacobian(result)
@test DiffResults.jacobian(result3) == DiffResults.jacobian(result)

sy = @SVector fill(zero(eltype(fx)), 2)
fresult1 = DiffResults.JacobianResult(sy, fx)
fresult2 = DiffResults.JacobianResult(sy, fx)
fresult3 = DiffResults.JacobianResult(sy, fx)
fresult1 = ForwardDiff.jacobian!(fresult1, diff, fx)
fresult2 = ForwardDiff.jacobian!(fresult2, diff, fx, cfg)
fresult3 = ForwardDiff.jacobian!(fresult3, diff, fx, fcfg)
@test DiffResults.value(fresult1) == DiffResults.value(result)
@test DiffResults.value(fresult2) == DiffResults.value(result)
@test DiffResults.value(fresult3) == DiffResults.value(result)
@test DiffResults.jacobian(fresult1) == DiffResults.jacobian(result)
@test DiffResults.jacobian(fresult2) == DiffResults.jacobian(result)
@test DiffResults.jacobian(fresult3) == DiffResults.jacobian(result)

  ...testing specialized FieldVector codepaths


LoadError: [91mUndefVarError: @test not defined[39m